In [2]:
from io import open
import os
from conllu import parse
from tqdm import tqdm
import pandas as pd
import re

In [3]:
file = "light_dev.CONLLUA"

In [21]:
data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]

In [5]:
df = pd.DataFrame(sentence)
# df

In [6]:
df = df[['form','identity']]
df.index = df.index + 1

In [7]:
entity = {}

open_stack_index = []
stack_token = []

data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]
            for index, token in enumerate(sentence):
                identities = token['identity']
                for identity in identities:
                    if "(" in identity:
                        stack_token.append(str(token))
                        open_stack_index.append(index)
                    elif ")" in identity:
                        count = str(identity).count(")")
                        end_index = index
                        for i in range(0, count):
                            if len(open_stack_index) == 0:
                                print("Do not match !")
                            else:
                                start_index = open_stack_index.pop()
                                mention = ""
                                for i in range(start_index, end_index+1):
                                    mention += sentence[i]["form"] + " "
                                mention = mention.strip()
                                if mention not in entity:
                                    entity[mention] = len(entity)
            data = """"""

In [8]:
entities = list(entity.keys())
# entities

In [9]:
entity_df = pd.DataFrame({"entity": entities}, dtype=object)
# entity_df

In [10]:
entity_df['index'] = ['0']*len(entity_df)
entity_df['head'] = ['0']*len(entity_df)
# entity_df

In [11]:
token_df = df.copy()
# token_df.head(10)

In [188]:
# entity id 

# entity = 'a ceiling'
# entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
# # print(entity_index)

# idx_li = []
# for token in entity.split(" "):
#     index = token_df.index[(token_df['form']==token)].tolist()[0]
#     idx_li.append(index)    

# print(idx_li)
# entity_df.loc[entity_index, 'index'] = idx_li
# token_df = token_df.drop(index=idx_li, axis=0)

In [14]:
# unused_token = token_df.index[token_df['identity'].apply(lambda x:x == ['_'])].tolist()
# token_df[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']=='a')]

In [15]:
# len(unused_token)

In [16]:
# token_df = token_df.drop(index=unused_token, axis=0)
# token_df

In [12]:
# 중복 entity 구분 X
identity_li = token_df.identity.tolist()

identities = []   
token_index = []

temp_id = []
temp_tk = []

l_count = 0
r_count = 0
for index, identity in enumerate(identity_li):
    l_count += ''.join(identity).count('(')
    r_count += ''.join(identity).count(')')
    
    if l_count == r_count and identity == ['_']:
        continue
        
    temp_id += identity
    temp_tk.append(index+1)

    if l_count == r_count:
        identities.append(''.join(temp_id))
        token_index.append(temp_tk)            

        temp_id = []
        temp_tk = []

In [13]:
len(identities)
len(token_index)
# token_index

2894

In [14]:
# token_index

In [15]:
new_df = pd.DataFrame(columns=['entity', 'index', 'identity', 'head'])
new_df

,entity,index,identity,head


In [16]:
new_df['index'] = token_index
new_df['identity'] = identities
new_df.index = new_df.index+1
new_df.head(10)

,entity,index,identity,head
1,NaN,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,NaN,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,NaN,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,NaN,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,NaN,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,NaN,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,NaN,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,NaN,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,NaN,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,NaN,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [17]:
new_df['entity'] = ['0']*len(new_df)

In [18]:
new_df.head(20)

,entity,index,identity,head
1,0,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,0,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,0,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,0,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,0,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,0,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,0,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,0,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,0,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,0,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [19]:
# sentences

In [22]:
sentences[0][0]['form']

'a'

In [23]:
index_li = new_df['index'].tolist()

for index, entity_idx in enumerate(index_li):
    
    temp_token = []
    for token_idx in entity_idx:
        temp_token.append(sentences[0][token_idx-1]['form'])
        
    new_df.loc[index+1, 'entity'] = ' '.join(temp_token)

In [24]:
new_df.head(10)

,entity,index,identity,head
1,a ceiling,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,a tree,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,a floor,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,a bag,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,a shield,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,a sword,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,an armor,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,a shoes,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,a vest,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,member,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [25]:
new_df.head(20)

,entity,index,identity,head
1,a ceiling,"[1, 2]",(EntityID=1MarkableID=markable_1Min=2SemType=dn),NaN
2,a tree,"[3, 4]",(EntityID=2MarkableID=markable_2Min=4SemType=dn),NaN
3,a floor,"[5, 6]",(EntityID=3MarkableID=markable_3Min=6SemType=dn),NaN
4,a bag,"[7, 8]",(EntityID=4MarkableID=markable_4Min=8SemType=dn),NaN
5,a shield,"[9, 10]",(EntityID=5MarkableID=markable_5Min=10SemType=dn),NaN
6,a sword,"[11, 12]",(EntityID=6MarkableID=markable_6Min=12SemType=dn),NaN
7,an armor,"[13, 14]",(EntityID=7MarkableID=markable_7Min=14SemType=dn),NaN
8,a shoes,"[15, 16]",(EntityID=8MarkableID=markable_8Min=16SemType=dn),NaN
9,a vest,"[17, 18]",(EntityID=9MarkableID=markable_9Min=18SemType=dn),NaN
10,member,[19],(EntityID=10MarkableID=markable_10Min=19SemTyp...,NaN


In [26]:
# entity_df.entity

In [27]:
# new_df.entity

In [28]:
# 중복 entity 분리
identity_li = token_df.identity.tolist()

identities = []   
token_index = []

temp_id = []
temp_tk = []

l_count = 0
r_count = 0
for index, identity in enumerate(identity_li):
#     print('index, identity:', index+1, identity)
    l_count += ''.join(identity).count('(')
    r_count += ''.join(identity).count(')')
#     print('l_count:', l_count)
#     print('r_count:', r_count)
    
    if l_count == r_count and identity == ['_']:
        l_count = 0
        r_count = 0
        continue
        
    temp_id += identity
    temp_tk.append(index+1)

    if l_count == r_count:   # 중복 X
        if l_count == 1:
            identities.append(''.join(temp_id))
            token_index.append(temp_tk)      
#             print('identities:', identities)
#             print('token_index:', token_index)

            temp_id = []
            temp_tk = []
            l_count = 0
            r_count = 0
            
        else:   # 중복
            redundant_id = ''.join(temp_id)
#             print('temp index:', temp_id)
            for i in range(l_count):
                temp_redundant_id = redundant_id[redundant_id.rindex('('):redundant_id.index(')')+1]
                redundant_id = redundant_id.replace(temp_redundant_id,'')
                identities.append(temp_redundant_id)
#                 token_index.append(temp)
                
#                 print('***redundant***')
#                 print('temp redundant id:', temp_redundant_id)
#                 print('identities:', identities)
                
#     print('='*50)
  

In [29]:
redundant_id = '(EntityID=13MarkableID=markable_47Min=50SemType=do(EntityID=11MarkableID=markable_136Min=49SemType=doElementOf=23))'
# print(redundant_id.rindex('('))
l_count = 2
identities = []
for i in range(l_count):
    temp_redundant_id = redundant_id[redundant_id.rindex('('):redundant_id.index(')')+1]
    redundant_id = redundant_id.replace(temp_redundant_id,'')
#     print(redundant_id)
#     redundant_id = redundant_id[:redundant_id.rindex('(')]

In [31]:
# # 중복 entity
# for index, identity in enumerate(identities):
#     if identity.count('(') > 1 and identity.count(')') > 1:
# #         print(identity)
# #         print(new_df.loc[index, 'entity'])

In [32]:
# index_total = []
# for index, entity in enumerate(entities):
#     entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
#     print("entity:", entity)

#     idx_li = []
#     identity = []
#     for token in entity.split(" "):
#         print(token)
# #         print(token_df.index[(token_df['form']==token)].tolist()[0]) # entity에 해당하는 token의 인덱스
        
#         # token 중에서 unused_token인 것은 제외 (entity로 사용되지 않은 token 제외)
# #         index = token_df.index[(token_df['form']==token)].tolist()[0]
#         index = token_df.index[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']==token)].tolist()[0]

#         idx_li.append(index)
#         index_total.append(index)
#         identity += token_df.loc[index,'identity']
    
#     print('token index:', idx_li)
    
#     identity_to_string = ''.join(identity)
#     print('identity:',identity_to_string)
#     print("="*50)

In [33]:
# index_total = []
# for index, entity in enumerate(entities):
#     entity_index = entity_df.index[(entity_df['entity']==entity)].tolist()[0]
#     print("entity:", entity)

#     idx_li = []
#     identity = []
#     for token in entity.split(" "):
#         print(token)
# #         print(token_df.index[(token_df['form']==token)].tolist()[0]) # entity에 해당하는 token의 인덱스
        
#         # token 중에서 unused_token인 것은 제외 (entity로 사용되지 않은 token 제외)
# #         index = token_df.index[(token_df['form']==token)].tolist()[0]
#         index = token_df.index[(token_df['identity'].apply(lambda x:x != ['_'])) & (token_df['form']==token)].tolist()[0]

#         idx_li.append(index)
#         index_total.append(index)
#         identity += token_df.loc[index,'identity']
    
#     print('token index:', idx_li)
    
#     identity_to_string = ''.join(identity)
#     print('identity:',identity_to_string)
    
#     if identity_to_string.count('(') == 1 and identity_to_string.count(')') == 1:  # 중복되지 않는 entity
#         identity_head = identity_to_string
#         print(identity_head)
#         token_df = token_df.drop(index=idx_li, axis=0)
# #         token_df = token_df[token_df.index > idx_li[-1]]  
#         print('deleted')
    
#     elif identity_to_string.count('(') != identity_to_string.count(')'):  # entity 안에 있는 entity
# #         token_df = token_df[token_df.index > idx_li[0]-1]
#         identity_head = identity_to_string[identity_to_string.rindex('('):identity_to_string.index(')')+1]
#         print(identity_head)
        
#     else:
#         identity_head = identity_to_string[:identity_to_string.rindex('(')]  # 큰 entity
#         print(identity_head)
# #         token_df = token_df[token_df.index > idx_li[-1]]
#         token_df = token_df.drop(index=idx_li, axis=0)    
#         print("deleted")
        
        
#     # head
#     pattern = re.compile(r"Min=\d+[,]{0,1}\d*[,]{0,1}\d*")
#     head= re.findall(pattern, identity_head)[0]
#     head_id = head.replace("Min=","")

# #     entity_idx = entity_df.index[entity_df['entity']==entity].tolist()[0]
#     entity_df.loc[entity_index, 'head'] = head_id
    
    
#     print("="*20)
# # token_df

In [34]:
# entity_df.head(50)

In [35]:
identity_head = "(EntityID=12MarkableID=markable_108Min=66,67SemType=dn"
pattern = re.compile(r"Min=\d+[,]{0,1}\d*[,]{0,1}\d*")
head= re.findall(pattern, identity_head)[0]
head

'Min=66,67'

In [36]:
head_id = head.replace("Min=","")
head_id

'66,67'

In [37]:
entity = 'a ceiling'
entity_idx = entity_df.index[entity_df['entity']==entity].tolist()[0]
entity_df.loc[entity_idx, 'head'] = head_id

In [38]:
# entity_df

In [39]:
# head

entity = {}

open_stack_index = []
stack_token = []

head = []
data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]
            for index, token in enumerate(sentence):
                identities = token['identity']
                for identity in identities:
                    if "(" in identity:
                        stack_token.append(str(token))
                        open_stack_index.append(index)
                    elif ")" in identity:
                        count = str(identity).count(")")
                        end_index = index
                        for i in range(0, count):
                            if len(open_stack_index) == 0:
                                print("Do not match !")
                            else:
                                start_index = open_stack_index.pop()
                                for i in range(start_index, end_index+1):
#                                     print(sentence[i]['identity'])
                                    if len(sentence[i]['identity']) == 1:
                                        continue
                                    head.append(sentence[i]["identity"])
#                                 head = head.strip()
#                                 if head not in entity:
#                                     entity[head] = len(entity)
#             data = """"""

KeyboardInterrupt: 

In [ ]:
# # head
# for h in head:
#     stringh = ''.join(h)
#     print(h)
#     print(stringh)
    
#     print("(:", stringh.count('('))
#     print("):", stringh.count(')'))
#     print("-"*20)

In [ ]:
# en = ['(EntityID=13','MarkableID=markable_47','Min=50','SemType=do(EntityID=11', 'MarkableID=markable_136','Min=49','SemType=do','ElementOf=23)']
# stringen = ''.join(en)
# print(stringen)
# l = re.findall('\(([^)]+)',stringen)
# print(l)
# l2 = re.findall('\(([^)]+)',l[0])
# print(l2)

In [390]:
# entity_df['head'] = ['0']*len(entity_df)
# entity_df

,entity,index,head
0,a ceiling,"[1, 2]",0
1,a tree,0,0
2,a floor,0,0
3,a bag,0,0
4,a shield,0,0
...,...,...,...
1569,the sick,0,0
1570,the lepers,0,0
1571,these golden halls,0,0
1572,you words,0,0


In [392]:
len(head)

1864081

In [ ]:
# # head

# token = 'a'
# index = df.index[(df['form']==token)].tolist()[0]
# column_identity = df.loc[index]['identity']
# print(column_identity)
# # print(len(column_identity))
# head_id = column_identity[2].replace("Min=","")
# # head_id

# df.loc[index, 'head'] = head_id
# # df = df.drop(index = index)
# df

In [ ]:
# for idx in range(1,token_df.shape[0]+1):
#     column_identity = token_df.loc[idx]['identity']
#     print(column_identity)
# #     if len(column_identity) < 3:
# #         continue
# #     head_id = column_identity[2].replace("Min=","")
# #     token_df.loc[index, 'head'] = head_id
# # token_df

In [ ]:
# for index, token in enumerate(sentence):
    
#     index = df.index[(df['form']==token)].tolist()[0]
#     column_identity = df.loc([index])['identity']
    
#     if len(column_identity) != 1:
#         head_id = column_identity[2].replace('Min=','')
#         df.loc[index, 'head'] = head_id